In [ ]:
!pip install pandas; numpy; psycopg2

In [ ]:
# importing needed libraries
import pandas as pd
import numpy as np
import psycopg2
from datetime import datetime, timedelta
import csv

In [ ]:
# reading the dataset from the source(CSV)
zulo_bank_df = pd.read_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\zulo_bank.csv')

In [ ]:
zulo_bank_df.head()

In [ ]:
zulo_bank_df.info()

In [ ]:
zulo_bank_df ['OpeningDate'] = pd.to_datetime(zulo_bank_df['OpeningDate'])
zulo_bank_df ['StartDate'] = pd.to_datetime(zulo_bank_df['StartDate'])
zulo_bank_df ['EndDate'] = pd.to_datetime(zulo_bank_df['EndDate'])
zulo_bank_df ['TransactionDate'] = pd.to_datetime(zulo_bank_df['TransactionDate'])

In [ ]:
zulo_bank_df.isnull().sum()

In [ ]:
# Filling the null values 
zulo_bank_df['LoanAmount'] = zulo_bank_df['LoanAmount'].fillna(0)
zulo_bank_df['LoanType'] = zulo_bank_df['LoanType'].fillna('Not Provided')
zulo_bank_df['InterestRate'] = zulo_bank_df['InterestRate'].fillna(0)

In [ ]:
# Normalization
zulo_bank_df [['First_Name', 'Last_Name']] = zulo_bank_df['FullName'].str.split(expand=True)

In [ ]:

transaction_df = zulo_bank_df[['TransactionType', 'Amount','TransactionDate']].copy().drop_duplicates().reset_index(drop =True)
transaction_df['TransactionID'] = range(1, len(transaction_df) + 1)
transaction_df = transaction_df[['TransactionID', 'TransactionType', 'Amount','TransactionDate']]

In [ ]:
customer_df = zulo_bank_df [['First_Name', 'Last_Name', 'Email', 'Phone']].copy().drop_duplicates().reset_index(drop = True)
customer_df['CustomerID'] = range(1, len(customer_df) + 1)
customer_df = customer_df[['CustomerID','First_Name', 'Last_Name', 'Email', 'Phone']]

In [ ]:
account_df = zulo_bank_df[['AccountType', 'Balance', 'OpeningDate']].copy().drop_duplicates().reset_index(drop =True)
account_df['AccountID'] = range(1, len(account_df)+ 1)
account_df = account_df[['AccountID', 'AccountType', 'Balance', 'OpeningDate']]

In [ ]:
loan_df = zulo_bank_df[['LoanAmount', 'LoanType', 'StartDate', 'EndDate', 'InterestRate']]. copy().drop_duplicates().reset_index(drop =True)
loan_df['LoanID'] = range(1, len(loan_df) + 1)
loan_df = loan_df[['LoanID','LoanAmount', 'LoanType', 'StartDate', 'EndDate', 'InterestRate']]

In [ ]:
# Creating a comprehensive zulo_bank df with the new IDs
zulo_bank_df = zulo_bank_df.merge(customer_df, on  =['First_Name', 'Last_Name', 'Email', 'Phone'], how = 'left')\
                            .merge(account_df, on= ['AccountType', 'Balance', 'OpeningDate'], how = 'left')\
                            .merge(loan_df, on = ['LoanAmount', 'LoanType', 'StartDate', 'EndDate', 'InterestRate'], how = 'left')\
                            .merge(transaction_df, on = ['TransactionType', 'Amount','TransactionDate'])\
                            [['CustomerID', 'AccountID', 'LoanID','TransactionID']]

In [ ]:
zulo_bank_df.info()

In [ ]:
#Creating a date table for all date columns to avoid transititive dependency
start_date = datetime(2020, 1, 1)
current_date= datetime(2050, 12, 31)

#number of days between start date and current date
num_days =(current_date - start_date).days

#getting a list of dates from start date to the current date
date_list = [start_date + timedelta(days =x) for x in range(num_days+1)] # why time delta and not datetime

#Ensuring date_id matches the length of the date_df
date = {'DateID':[x for x in  range(1, len(date_list) + 1)], 'date': date_list}

date_dim = pd.DataFrame(date)
date_dim['Day'] = date_dim['date'].dt.day
date_dim['Month'] = date_dim['date'].dt.month
date_dim['Year'] = date_dim['date'].dt.year


In [ ]:
date_dim.head()

In [ ]:
account_df = account_df.merge(date_dim, left_on = 'OpeningDate', right_on = 'date', how = 'inner')\
                        .rename(columns = {'DateID': 'OpeningDateID'})\
                        .reset_index (drop = True)\
                        [['AccountID', 'AccountType', 'Balance', 'OpeningDateID']]

In [ ]:
transaction_df = transaction_df.merge(date_dim, left_on ='TransactionDate', right_on = 'date', how = 'inner')\
                                .rename(columns = {'DateID' : 'TransactionDateID'})\
                                .reset_index (drop =True)\
                                [['TransactionID', 'TransactionType', 'Amount','TransactionDateID']]


In [ ]:
loan_df = loan_df.merge(date_dim, left_on = 'StartDate', right_on = 'date', how = 'inner')\
                        .rename(columns = {'DateID' : 'StartDateID'})\
                        .merge(date_dim, left_on = 'EndDate', right_on = 'date', how = 'inner')\
                        .rename(columns = {'DateID' : 'EndDateID'})\
                        .reset_index (drop =True)\
                        [['LoanID', 'LoanAmount', 'LoanType', 'StartDateID', 'EndDateID','InterestRate']]


In [ ]:
# zulo_bank_df = zulo_bank_df.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\zulo_fact.csv', index = False)

In [ ]:
#Creating models for my DWH
# transaction fact table
#loan fact table
transaction_dim = transaction_df[['TransactionID','TransactionType']].copy().drop_duplicates().reset_index(drop =True)

account_dim = account_df [['AccountID', 'AccountType', 'Balance']].copy().drop_duplicates().reset_index(drop =True)

transaction_fact = zulo_bank_df.merge(transaction_df, on = 'TransactionID', how = 'inner')\
                                .merge(account_df, on = 'AccountID', how = 'inner')\
                                [['TransactionID', 'AccountID', 'TransactionDateID', 'OpeningDateID', 'Amount']]



In [ ]:
loan_dim = loan_df[['LoanID', 'LoanType']].copy().drop_duplicates().reset_index(drop = True)
customer_dim = customer_df[['First_Name', 'Last_Name', 'Email', 'Phone']].copy().drop_duplicates().reset_index(drop = True)
loan_fact = zulo_bank_df.merge(loan_df, on = 'LoanID', how = 'inner')\
                        .merge(customer_df, on = 'CustomerID', how = 'inner')\
                        [['LoanID', 'CustomerID', 'StartDateID', 'EndDateID', 'LoanAmount', 'InterestRate']]

In [ ]:
transaction_dim = transaction_dim.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\transaction_dim.csv', index = False)
account_dim = account_dim.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\account_dim.csv', index = False)
loan_dim = loan_dim.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\loan_dim.csv', index = False)
customer_dim = customer_dim.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\customer_dim.csv', index = False)
loan_fact = loan_fact.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\loan_fact.csv', index = False)
transaction_fact = transaction_fact.to_csv(r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\transaction_fact.csv', index = False)

In [ ]:
# loading my data into the database
db_params = {
    'user': 'postgres',
    'password':'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'zulobank'
}

default_db_url = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/postgres"
try:
    conn = psycopg2.connect(default_db_url)
    conn.autocommit = True
    cursor = conn.cursor()
    cursor.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = '{db_params['database']}'")
    exists = cursor.fetchone()
    if not exists:
        cursor.execute(f'CREATE DATABASE {db_params['database']}')
        print (f'Database {db_params['database']} creted successfully')
    else:
        print(f'Database {db_params['database']} already exists')
    cursor.close()
    conn.close()

except Exception as e:
    print(f'An error has occurred: {e}')


In [ ]:

def get_db_connection():
    connection = psycopg2.connect(
    host = 'localhost',
    database = 'zulobank',
    user = 'postgres',
    password ='postgres'
    )
    return connection
conn = get_db_connection()


In [ ]:
# creating the required schema and tables
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    table_query = '''
                CREATE SCHEMA IF NOT EXISTS zulobankdb; 
                DROP TABLE IF EXISTS zulobankdb.customers CASCADE;
                DROP TABLE IF EXISTS zulobankdb.accounts CASCADE;
                DROP TABLE IF EXISTS zulobankdb.transactions CASCADE;
                DROP TABLE IF EXISTS zulobankdb.loans CASCADE;
                DROP TABLE IF EXISTS zulobankdb.date_dim CASCADE;
                DROP TABLE IF EXISTS zulobankdb.zulo_fact CASCADE;


                CREATE TABLE IF NOT EXISTS zulobankdb.date_dim(
                DateID SERIAL PRIMARY KEY,
                date VARCHAR (1000),
                Day INT,
                Month INT,
                Year INT
                
                );

                CREATE TABLE IF NOT EXISTS zulobankdb.transactions(
                TransactionID SERIAL PRIMARY KEY,
                TransactionType VARCHAR (1000),
                Amount FLOAT,
                TransactionDateID INT,
                FOREIGN KEY(TransactionDateID) REFERENCES zulobankdb.date_dim(DateID)
                
                );


                CREATE TABLE IF NOT EXISTS zulobankdb.accounts(
                AccountID SERIAL PRIMARY KEY,
                AccountType VARCHAR (1000),
                Balance FLOAT,
                OpeningDateID INT,
                FOREIGN KEY(OpeningDateID) REFERENCES zulobankdb.date_dim(DateID)
                
                );

                CREATE TABLE IF NOT EXISTS zulobankdb.loans(
                LoanID SERIAL PRIMARY KEY,
                LoanAmount FLOAT,
                LoanType VARCHAR(1000),
                StartDateID INT,
                EndDateID INT,
                InterestRate FLOAT,
                FOREIGN KEY(StartDateID) REFERENCES zulobankdb.date_dim(DateID),
                FOREIGN KEY(EndDateID) REFERENCES zulobankdb.date_dim(DateID)
                
                );


                CREATE TABLE IF NOT EXISTS zulobankdb.customers(
                CustomerID SERIAL PRIMARY KEY,
                First_Name VARCHAR(1000),
                Last_Name VARCHAR(1000),
                Email VARCHAR(1000),
                Phone VARCHAR(1000)
                );
            
                CREATE TABLE IF NOT EXISTS zulobankdb.zulo_fact(
                CustomerID INT,
                AccountID INT,
                LoanID INT,
                TransactionID INT
                );'''
    cursor.execute(table_query)
    conn.commit()
    cursor.close()
    conn.close()

 

In [ ]:
create_tables()

In [ ]:
#loading the datasets into the created db tables
def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' INSERT INTO  zulobankdb.date_dim (DateID, date, Day, Month, Year) 
                           VALUES (%s, %s, %s, %s, %s);''', 
                           row)
            
    conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\date_dim.csv'
load_data_from_csv(csv_path)
print('date_dim data loaded successfully')



In [ ]:

def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' INSERT INTO  zulobankdb.transactions (TransactionID, TransactionType, Amount, TransactionDateID) 
                           VALUES (%s, %s, %s, %s);''', 
                           row)
            
    conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\transaction.csv'
load_data_from_csv(csv_path)
print('transactions data loaded successfully')
        

In [ ]:

def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' INSERT INTO  zulobankdb.accounts (AccountID, AccountType, Balance, OpeningDateID) 
                           VALUES (%s, %s, %s, %s);''', 
                           row)
            
    conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\account.csv'
load_data_from_csv(csv_path)
print('accounts data loaded successfully')

In [ ]:

def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' INSERT INTO  zulobankdb.loans (loanID, LoanAmount, LoanType, StartDateID, EndDateID, InterestRate) 
                           VALUES (%s, %s, %s, %s, %s, %s);''', 
                           row)
            
    conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\loan.csv'
load_data_from_csv(csv_path)
print('loans data loaded successfully')

In [ ]:

def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(''' INSERT INTO  zulobankdb.customers (CustomerID, First_Name, Last_Name, Email, Phone) 
                           VALUES (%s, %s, %s, %s, %s);''', 
                           row)
            
    conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\customer.csv'
load_data_from_csv(csv_path)
print('customers data loaded successfully')

In [ ]:

def load_data_from_csv(csv_path):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            try:
                cursor.execute(''' INSERT INTO  zulobankdb.zulo_fact (CustomerID, AccountID, LoanID, TransactionID) 
                           VALUES (%s, %s, %s, %s);''', 
                           row)
        
                
            except psycopg2.IntegrityError as e: # this is to aaddress the foreign key violation
                print ('error inserting row;', row)
                conn.rollback() # This rolls back the current transaction
            else:
                conn.commit()
    cursor.close()
    conn.close()

csv_path = r'C:\Users\Admin\Desktop\10Alytics\Zulo Bank Data Model\Zulo_Bank_Project\Datasets\Transformed_Datasets\zulo_fact.csv'
load_data_from_csv(csv_path)
print('zulo_fact data loaded successfully')